In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from eo_tools.S1_dev import S1IWSwath
from eo_tools.S1_dev import presum, align, coregister, presum
from eo_tools.util import show_cog
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt
import numpy as np

from folium import LayerControl
from eo_tools.S1_dev import boxcar


In [ ]:
data_dir = "/data/S1"
master_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230903T183344_20230903T183412_050167_0609B4_100E.SAFE"
slave_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230915T183345_20230915T183413_050342_060F9F_85A4.SAFE"
iw = 1
pol = "vv"
up = 2

In [ ]:
ifgs = []
lut = []
dems = []

mst = S1IWSwath(master_dir, iw=iw, pol=pol)
slv = S1IWSwath(slave_dir, iw=iw, pol=pol)
overlap = np.round(mst.compute_burst_overlap(2)).astype(int)

boxsiz = 5

def avg_ampl(arr, boxsiz):
    return np.sqrt(boxcar(np.nan_to_num(abs(arr)**2, 1e-10), boxsiz, boxsiz))

for burst_idx in range(1, 11):
    print(f"---- Processing burst {burst_idx} ----")

    # compute geocoding LUTs for master and slave bursts
    file_dem = mst.fetch_dem_burst(burst_idx, dir_dem="/data/tmp", force_download=False)
    az_mst, rg_mst = mst.geocode_burst(file_dem, burst_idx=burst_idx, dem_upsampling=up)
    az_slv, rg_slv = slv.geocode_burst(file_dem, burst_idx=burst_idx, dem_upsampling=up)

    # read burst raster
    arr_mst = mst.read_burst(burst_idx, True)
    arr_slv = slv.read_burst(burst_idx, True)

    # deramp slave
    pdb_slv = slv.deramp_burst(burst_idx)
    arr_slv_de = arr_slv * np.exp(1j * pdb_slv).astype(np.complex64)

    # project slave LUT into master grid
    # print("coreg_fast")
    az_co, rg_co = coregister(arr_mst, az_mst, rg_mst, az_slv, rg_slv)

    # warp raster slave and deramping phase
    arr_slv_re = align(arr_mst, arr_slv_de, az_co, rg_co, order=3)
    pdb_slv_re = align(arr_mst, pdb_slv, az_co, rg_co, order=3)

    # reramp slave
    # arr_slv_re_de = arr_slv_re * np.exp(-1j * pdb_slv_re).astype(np.complex64)
    arr_slv_re = arr_slv_re * np.exp(-1j * pdb_slv_re).astype(np.complex64)

    # compute topographic phases
    rg_mst_rdr = np.zeros(arr_slv.shape[0])[:, None] + np.arange(0, arr_slv.shape[1])
    pht_mst = mst.phi_topo(rg_mst_rdr).reshape(*arr_mst.shape)
    pht_slv = slv.phi_topo(rg_co.ravel()).reshape(*arr_mst.shape)
    # phi_topo = pht_mst - pht_slv
    pha_topo = np.exp(-1j * (pht_mst - pht_slv)).astype(np.complex64)

    # interferogram without topographic phase
    ifg = (arr_slv_re * arr_mst.conj() * pha_topo).conj()

    # normalized complex coherences  
    pows = avg_ampl(arr_mst, boxsiz) * avg_ampl(arr_slv_re, boxsiz)
    ifgs.append(boxcar(np.nan_to_num(ifg), boxsiz, boxsiz) / pows)
    lut.append((az_mst, rg_mst))
    dems.append(file_dem)

In [ ]:
from eo_tools.S1_dev import fast_esd
fast_esd(ifgs, overlap)

In [ ]:
from eo_tools.S1_dev import stitch_bursts
img = stitch_bursts(ifgs, overlap)

In [ ]:
from eo_tools.S1_dev import resample
from eo_tools.util import palette_phi
from folium import Map
import rioxarray as riox
from rioxarray.merge import merge_arrays

# m = Map()
off = 0
H = int(overlap / 2)
phi_out = presum(np.nan_to_num(img), 2, 8)
naz = ifgs[0].shape[0]
list_phi = []
list_coh = []
for i in range(len(ifgs)):
    az_mst, rg_mst = lut[i]
    file_dem = dems[i]
    cnd = (az_mst >= H - 4) & (az_mst < naz - H + 4)
    az_mst2 = az_mst.copy()
    rg_mst2 = rg_mst.copy()
    az_mst2[~cnd] = np.nan
    rg_mst2[~cnd] = np.nan

    file_phi = f"/data/res/test_remap_burst_{i}_phi.tif"
    file_coh = f"/data/res/test_remap_burst_{i}_coh.tif"

    if i == 0:
        off2 = off
    else:
        off2 = off - H
    resample(
        np.abs(phi_out),
        file_dem,
        file_coh,
        (az_mst2 + off2) / 2,
        (rg_mst2) / 8,
        order=3,
        write_phase=False,
    )
    resample(
        phi_out,
        file_dem,
        file_phi,
        (az_mst2 + off2) / 2,
        (rg_mst2) / 8,
        order=3,
        write_phase=True,
    )
    if i == 0:
        off += naz - H
    else:
        off += naz - 2 * H

    # hack to avoid GDAL caching
    scale_phi = np.pi + np.random.rand() * 1e-6
    rnd_coh = np.random.rand() * 1e-6

    # show_cog(file_phi, m, rescale=f"-{scale_phi:.8f},{scale_phi:8f}", colormap=palette_phi())
    # show_cog(file_coh, m, rescale=f"{0+rnd_coh:.8f}, {1+rnd_coh:.8f}")
    list_phi.append(riox.open_rasterio(file_phi))
    list_coh.append(riox.open_rasterio(file_coh))

merged_phi = merge_arrays(list_phi)
merged_coh = merge_arrays(list_coh)
merged_phi.rio.to_raster("/data/res/test_merge_phi.tif")
merged_coh.rio.to_raster("/data/res/test_merge_coh.tif")

# LayerControl().add_to(m)
# m

In [ ]:
m = Map()
show_cog("/data/res/test_merge_coh.tif", m,rescale=f"0,1.01")
show_cog("/data/res/test_merge_phi.tif", m,rescale=f"-3,3.01",colormap=palette_phi())
LayerControl().add_to(m)
m

In [ ]:
from eo_tools.S1_dev import resample
from eo_tools.util import palette_phi
from folium import Map

m = Map()
for i in range(len(ifgs)):
    az_mst, rg_mst = lut[i]
    file_dem = dems[i]
    # pht = phts[i]
    # phi_out = presum(np.nan_to_num(ifgs[i]*np.exp(1j * pht)), 2, 8)
    # crop in the geocoded domain
    cnd = (az_mst >= int(overlap/2) - 4) & (az_mst <= ifgs[i].shape[0] - 1 - int(overlap/2) + 4)
    az_mst2 = az_mst.copy()
    rg_mst2 = rg_mst.copy()
    az_mst2[~cnd] = np.nan
    rg_mst2[~cnd] = np.nan
    phi_out = presum(np.nan_to_num(ifgs[i]), 2, 8)
    file_out = f"/data/res/test_remap_burst_ml_{i}.tif"
    resample(
        phi_out,
        file_dem,
        file_out,
        az_mst2 / 2,
        rg_mst2 / 8,
        order=3,
        write_phase=True,
    )

    # hack to avoid GDAL caching
    scale = np.pi + np.random.rand() * 1e-6

    show_cog(file_out, m, rescale=f"-{scale:.8f},{scale:8f}", colormap=palette_phi())

LayerControl().add_to(m)
m